### Analysis of connexin coverage of blood vessels

*Measurement "Blood vessel connexin coverage" of step (e) in the block diagram in the methods section."

In [ ]:
import os
import pickle
from pathlib import Path
from PIL import Image
import numpy as np
import scipy.ndimage as ndi
import pandas as pd

connexin_bin_path = Path('Cx43_connexin/binary')    # Segmented connexin
connexin_net_path = Path('Cx43_connexin/network')   # Graph generated from segmentation using pyvane
vessel_bin_path = Path('vessels/binary')            # Segmented blood vessels
vessel_net_path = Path('vassels/network')           # Graph generated from vessel segmentation using pyvane

In [ ]:
def draw_graph(graph, img_shape=None):
    '''Draw graph in an image.'''
    
    if img_shape is None:
        img_shape = graph.graph['shape']
    out_img = np.zeros(img_shape, dtype=int)

    for node, pixels in graph.nodes(data='pixels'):
        for pixel in pixels:
            out_img[tuple(pixel)] = 1

    for node, center in graph.nodes(data='center'):
        out_img[tuple(center)] = 1

    for _, _, path in graph.edges(data='path'):
        for pixel in path:
            out_img[tuple(pixel)] = 1
            
    return out_img

def draw_graph_indices(graph, img_shape=None):
    '''Draw indices of graph edges in an image.'''
    
    if img_shape is None:
        img_shape = graph.graph['shape']
    out_img = np.full(img_shape, -1, dtype=int)

    for edge_idx, (_, _, path) in enumerate(graph.edges(data='path')):
        for pixel in path:
            out_img[tuple(pixel)] = edge_idx
            
    return out_img

def calculate_radii(graph, img_bin):
    '''Estimate blood vessel radius.'''
    
    img_dist_inside = ndi.distance_transform_edt(img_bin)
    
    radii = {}
    for edge_idx, (_, _, path) in enumerate(graph.edges(data='path')):
        path_radii = []
        for pixel in path:
            path_radii.append(img_dist_inside[tuple(pixel)])
        radii[edge_idx] = np.median(path_radii)
        
    return radii

def read_data(file, connexin_bin_path, connexin_net_path, vessel_bin_path, vessel_net_path):
    '''Read files.'''
    
    filename_con = file.split('.')[0]
    
    img_con_bin = pickle.load(open(connexin_bin_path/f'{filename_con}.pickle', 'rb'))
    img_con_bin = img_con_bin.data
    graph_con = pickle.load(open(connexin_net_path/f'{filename_con}.pickle', 'rb'))
    filename_ves = filename_con.replace('C2', 'C4')
    try:
        img_ves_bin = np.array(Image.open(vessel_bin_path/f'{filename_ves}.png'))
    except FileNotFoundError:
        filename_ves = filename_con.replace('C2', 'C1')
        img_ves_bin = np.array(Image.open(vessel_bin_path/f'{filename_ves}.png'))
    img_ves_bin[img_ves_bin==255] = 1
    graph_ves = pickle.load(open(vessel_net_path/f'{filename_ves}.pickle', 'rb'))
    
    return img_con_bin, graph_con, img_ves_bin, graph_ves

def expand_skeleton_data(img_skel, img_skel_data, img_bin, outside_value=-1):
    '''For each white pixel in `img_bin`, assign the value of the closest pixel in `img_skel_data`.'''
    
    img_dist, ind_closest = ndi.distance_transform_edt(1-img_skel, return_indices=True)
    img_ves_ind = img_skel_data[ind_closest[0], ind_closest[1]]
    img_ves_ind[img_bin==0] = outside_value
    
    return img_ves_ind

vessel_dil = 13      # Distance for expanding the vessels. Connexing overlapping with the expanded vessels are considered to be in contact.
pix_size = 0.454     # Voxel size in micrometers

files = os.listdir(connexin_bin_path)
connexin_data = {}
for file in files:
    print(file)
    
    file_tag = '@'.join(file.split('@')[:-1])

    img_con_bin, graph_con, img_ves_bin, graph_ves = read_data(file, connexin_bin_path, connexin_net_path, 
                                                               vessel_bin_path, vessel_net_path)

    radii = calculate_radii(graph_ves, img_ves_bin)
    
    img_graph_con = draw_graph(graph_con)
    img_graph_ves_idx = draw_graph_indices(graph_ves)
    img_graph_ves = 1*(img_graph_ves_idx>-1)

    # Expand graph edges for overlap detection
    img_ves_mask = ndi.binary_dilation(img_ves_bin, iterations=vessel_dil)
    img_ves_ind = expand_skeleton_data(img_graph_ves, img_graph_ves_idx, img_ves_bin)
    img_ves_mask_ind = expand_skeleton_data(img_graph_ves, img_graph_ves_idx, img_ves_mask)
    
    graph_edge_data = list(graph_ves.edges(data=True))
    vessel_segments = ndi.find_objects(img_ves_mask_ind+1)   #img_ves_mask_ind starts at -1
    segment_data = {}
    # For each vessel graph edge, identifies the amount of connexin area and length in contact
    for idx, segment in enumerate(vessel_segments):
        sub_img_ves_ind = img_ves_ind[segment]
        sub_img_ves_mask_ind = img_ves_mask_ind[segment]
        sub_img_con_bin = img_con_bin[segment]
        sub_img_graph_con = img_graph_con[segment]
        vessel_seg = sub_img_ves_ind==idx
        vessel_mask_seg = sub_img_ves_mask_ind==idx
        connexin_in_vessel = sub_img_con_bin[vessel_mask_seg]
        connexin_skel_in_vessel = sub_img_graph_con[vessel_mask_seg]
        vessel_area = vessel_seg.sum()
        connexin_amount = connexin_in_vessel.sum()
        connexin_skel_amount = connexin_skel_in_vessel.sum()        
        
        segment_data[idx] = {
            'radius':radii[idx]*pix_size,
            'vessel_area':vessel_area*pix_size**2,
            'connexin_amount':connexin_amount*pix_size**2,
            'connexin_skel_amount':connexin_skel_amount*pix_size,
            'length':len(graph_edge_data[idx][2]['path'])*pix_size
        }
    
    connexin_data[file_tag] = segment_data

In [92]:
def find_index(value, values):
    
    ind = np.nonzero(value>values)[0]
    return ind[-1]

output_file = '../results/connexin_analysis.xlsx'

length_threshold = 3  # Very small edges are considered spurious
max_width = 20        # Very large edges are also considered spurious
width_thresholds = np.array([0, 4.5, max_width])

num_thresholds = len(width_thresholds)
results = {}
for file_tag, segment_data in connexin_data.items():
    vessel_areas = [[] for _ in range(num_thresholds)]
    vessel_lengths = [[] for _ in range(num_thresholds)]
    connexin_areas = [[] for _ in range(num_thresholds)]
    connexin_lengths = [[] for _ in range(num_thresholds)]
    for segment in segment_data.values():
        vessel_length = segment['length']
        if vessel_length>length_threshold:
            width = 2*segment['radius']
            ind = find_index(width, width_thresholds)
            vessel_areas[ind].append(segment['vessel_area'])
            vessel_lengths[ind].append(vessel_length)
            connexin_areas[ind].append(segment['connexin_amount'])
            connexin_lengths[ind].append(segment['connexin_skel_amount'])
           
    # Overlap between the two structures. Note that the values are for the overlap.
    results[file_tag] = [
        sum(connexin_areas[0])/sum(vessel_areas[0]),
        sum(connexin_areas[1])/sum(vessel_areas[1]),
        sum(connexin_lengths[0])/sum(vessel_lengths[0]),
        sum(connexin_lengths[1])/sum(vessel_lengths[1]),
        sum(connexin_areas[0]),
        sum(connexin_areas[1])
    ]

df = pd.DataFrame.from_dict(results, orient='index', columns=['connexin area/vessel area (width<=4.5 um)', # *Measurement used in the paper
                                                              'connexin area/vessel area (width>4.5 um)',  # *Measurement used in the paper
                                                              'connexin length/vessel length (width<=4.5 um)',
                                                              'connexin length/vessel length (width>4.5 um)',
                                                              'connexin area (width<=4.5 um)',
                                                              'connexin area (width>4.5 um)'
                                                             ])
df.to_excel(output_file, sheet_name='Results', index_label='Filename')